# Install the Package
Here we're installing it directly from GitHub while it's in development.

In [1]:
!pip install 'vanna[flask,anthropic]'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.2/388.2 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.5/485.5 kB 39.0 MB/s eta 0:00:00


# Download a Sample Database

In [2]:
import httpx

with open("Chinook.sqlite", "wb") as f:
    with httpx.stream("GET", "https://vanna.ai/Chinook.sqlite") as response:
        for chunk in response.iter_bytes():
            f.write(chunk)

# Imports

In [3]:
from vanna import Agent, AgentConfig
from vanna.servers.fastapi import VannaFastAPIServer
from vanna.core.registry import ToolRegistry
from vanna.core.user import UserResolver, User, RequestContext
from vanna.integrations.anthropic import AnthropicLlmService
from vanna.tools import RunSqlTool, VisualizeDataTool
from vanna.integrations.sqlite import SqliteRunner
from vanna.tools.agent_memory import SaveQuestionToolArgsTool, SearchSavedCorrectToolUsesTool
from vanna.integrations.local.agent_memory import DemoAgentMemory
from vanna.capabilities.sql_runner import RunSqlToolArgs
from vanna.tools.visualize_data import VisualizeDataArgs

# Define your User Authentication
Here we're going to say that if you're logged in as `admin@example.com` then you're in the `admin` group, otherwise you're in the `user` group

In [4]:
class SimpleUserResolver(UserResolver):
    async def resolve_user(self, request_context: RequestContext) -> User:
        # In production, validate cookies/JWTs here
        user_email = request_context.get_cookie('vanna_email')
        if not user_email:
            raise ValueError("Missing 'vanna_email' cookie for user identification")

        print(f"Resolving user for email: {user_email}")

        if user_email == "admin@example.com":
            return User(id="admin1", email=user_email, group_memberships=['admin'])

        return User(id="user1", email=user_email, group_memberships=['user'])

# Define the Tools and Access Control

In [5]:
tools = ToolRegistry()
tools.register_local_tool(RunSqlTool(sql_runner=SqliteRunner(database_path="./Chinook.sqlite")), access_groups=['admin', 'user'])
tools.register_local_tool(VisualizeDataTool(), access_groups=['admin', 'user'])
agent_memory = DemoAgentMemory(max_items=1000)
tools.register_local_tool(SaveQuestionToolArgsTool(), access_groups=['admin'])
tools.register_local_tool(SearchSavedCorrectToolUsesTool(), access_groups=['admin', 'user'])

In [ ]:
# Set up LLM
llm = AnthropicLlmService(model="claude-sonnet-4-5", api_key="sk-ant-...")


# Create agent with your options
agent = Agent(
    llm_service=llm,
    tool_registry=tools,
    user_resolver=SimpleUserResolver(),
    config=AgentConfig(),
    agent_memory=agent_memory
)

# 4. Create and run server
server = VannaFastAPIServer(agent)
server.run()